<a href="https://colab.research.google.com/github/upledge-general/finance-ai-community-public/blob/main/data_summarizer_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 Demo Notebook — Please Read First

Welcome! This is the exact notebook used in the live demo.

**How to use:**
1. Go to `Runtime` → `Run all`.
2. Wait until all cells finish (you’ll see ✅ messages).
3. Scroll and explore the outputs.

> 🔒 **Don't worry:** This original version is read-only.
> If you want to experiment, go to `File` → `Save a copy in Drive` and play in *your* copy.


# Data Summarizer with Advanced Insights

Turn raw datasets into **clear summaries, trends, drivers, and actionable insights** in just a few clicks.

This notebook is designed for **non-technical users**.  
You don’t need to edit any code.

---

## How this notebook works

1. Click **Runtime → Run all**
2. Start by uploading one of the **Sample Datasets** (recommended)
3. Select the **Dataset Type** (or keep it on Auto)
4. Click **▶ Generate insights**
5. Review the insights across the tabs
6. *(Optional)* Download the results

That’s it.

---

## What happens behind the scenes

When you run this notebook, it will automatically:

- Prepare the required environment  
- Load your uploaded dataset (or sample data)  
- Validate data quality and structure  
- Detect time, metrics, and dimensions automatically  
- Show which insights are available (via capability badges)  
- Generate:
  - Executive summary  
  - Trends over time (if applicable)  
  - Key drivers and contributors  
  - Anomaly signals (spikes and drops)  
- *(Optional)* Prepare an AI-ready summary for narrative generation  

If something needs attention, the notebook will show a **clear, human-readable message** explaining what’s missing or limited.

---

## First-time users (recommended)

For your first run, use one of the **Sample Datasets** provided.

This lets you:
- See the full insight generation flow
- Understand the outputs and tabs
- Learn how dataset structure affects insights

Once you’re comfortable, you can switch to **Upload Your Own Data** and re-run the analysis.

---

## Supported dataset types

You can choose a dataset type to improve validation and insights:

- **Business / Performance** – sales, revenue, cost, operations  
- **ESG / Impact** – ESG scores or sustainability metrics  
- **Startup / Deal Screening** – traction, growth, early-stage metrics  
- **Portfolio Monitoring** – portfolio company performance  
- **Generic (Auto)** – auto-detect structure when unsure  

If unsure, keep the selection on **Auto**.

---

## Supported input formats

- CSV (`.csv`)
- Excel (`.xlsx`, `.xls`)
- Parquet (`.parquet`)

---

## Insights controls (left panel)

You can fine-tune the analysis using simple controls:

- **Anomaly sensitivity** – how strict anomaly detection should be  
- **Top-N drivers** – how many top contributors to display  
- **Enable downloads** – allow or disable exporting results  

If you’re not sure, the **default settings work well** for most datasets.

---

## About the AI (LLM) feature

The AI narrative feature is **optional and disabled by default**.

When disabled:
- No external APIs are called  
- All analysis runs locally inside the notebook  

When enabled:
- The AI generates a plain-English summary  
- It uses **computed insights only**, not raw data  

You remain fully in control.

---

## Tip

This notebook is safe to explore.  
If you want to try again, simply upload a new file or re-run with a sample dataset.

No data is stored, and nothing is shared externally by default.


#### Install & Imports

In [1]:
# Install packages (once)
# uncomment this if you run for the first time
!pip install streamlit

print("✅ Setup complete")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0
✅ Setup complete


#### Web App

In [2]:
%%writefile app.py
# app.py — uPledge: Data Summarizer with Advanced Insights (V2.1)
# Adds user-friendly descriptions for every major section + each output tab.
# Run: streamlit run app.py

import io
import re
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import streamlit as st


# ============================
# Config
# ============================
ALLOWED_EXT = {"csv", "xlsx", "xls", "parquet"}
MAX_ROWS_PREVIEW_DEFAULT = 200
MAX_MB_SOFT = 60

COMMON_DATE_NAMES = {
    "date", "month", "period", "week", "year", "time", "timestamp",
    "dt", "order_date", "created_at", "start_date", "end_date"
}

COMMON_METRIC_HINTS = {
    "revenue", "sales", "volume", "units", "profit", "margin", "cost", "spend",
    "impressions", "clicks", "ctr", "cpc", "cpm", "roas",
    "co2", "carbon", "emissions", "energy", "water", "waste",
    "esg", "impact", "score", "rating", "users", "active", "retention",
    "valuation", "irr", "multiple", "ownership"
}

DATASET_OPTIONS = [
    "Auto",
    "Business/Performance",
    "ESG/Impact",
    "Startup/Deal Screening",
    "Portfolio Monitoring",
    "Generic",
]

DATASET_RULES = {
    "Business/Performance": {
        "time_candidates": ["date", "month", "period", "week"],
        "entity_candidates": ["company", "brand", "product", "customer"],
        "metric_keywords": ["revenue", "sales", "profit", "cost", "units", "volume", "spend"],
        "dims_keywords": ["region", "product", "channel", "category", "segment", "brand"],
    },
    "ESG/Impact": {
        "time_candidates": ["year", "date", "period"],
        "entity_candidates": ["company", "issuer", "entity"],
        "metric_keywords": ["e_score", "s_score", "g_score", "esg", "impact", "co2", "emissions", "energy", "water"],
        "dims_keywords": ["sector", "country", "pillar", "metric", "scope"],
    },
    "Startup/Deal Screening": {
        "time_candidates": ["date", "month", "period"],
        "entity_candidates": ["startup", "company", "venture"],
        "metric_keywords": ["users", "revenue", "gmv", "mrr", "arr", "growth", "retention", "burn", "runway"],
        "dims_keywords": ["sector", "country", "stage", "channel", "product"],
    },
    "Portfolio Monitoring": {
        "time_candidates": ["period", "date", "year", "quarter"],
        "entity_candidates": ["company", "portfolio_company", "asset"],
        "metric_keywords": ["valuation", "revenue", "irr", "multiple", "ownership", "ebitda"],
        "dims_keywords": ["fund", "sector", "region", "stage", "currency"],
    },
    "Generic": {
        "time_candidates": ["date", "month", "period", "year", "week"],
        "entity_candidates": [],
        "metric_keywords": [],
        "dims_keywords": [],
    },
    "Auto": {
        "time_candidates": ["date", "month", "period", "year", "week"],
        "entity_candidates": ["company", "startup", "product", "region"],
        "metric_keywords": [],
        "dims_keywords": [],
    },
}


# ============================
# Utilities
# ============================
def _clean_colname(c: str) -> str:
    return re.sub(r"\s+", " ", str(c)).strip()


def _file_ext(filename: str) -> str:
    return filename.lower().split(".")[-1].strip()


def _bytes_to_mb(n: int) -> float:
    return round(n / (1024 * 1024), 2)


def _to_numeric(s: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(s):
        return s
    return pd.to_numeric(s.astype(str).str.replace(",", "").str.strip(), errors="coerce")


def _try_parse_datetime_series(s: pd.Series) -> Tuple[pd.Series, float]:
    raw = s.astype(str).str.strip()
    raw = raw.replace({"": np.nan, "nan": np.nan, "None": np.nan, "NaT": np.nan})
    non_empty = raw.notna().sum()
    if non_empty == 0:
        return pd.to_datetime(pd.Series([pd.NaT] * len(s))), 0.0

    p1 = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)
    r1 = p1.notna().sum() / non_empty

    p2 = pd.to_datetime(raw, errors="coerce", format="%m-%Y")
    r2 = p2.notna().sum() / non_empty

    p3 = pd.to_datetime(raw, errors="coerce", format="%Y-%m")
    r3 = p3.notna().sum() / non_empty

    parsed, ratio = p1, r1
    for p, r in [(p2, r2), (p3, r3)]:
        if r > ratio:
            parsed, ratio = p, r

    return parsed, float(ratio)


def _safe_to_datetime(s: pd.Series) -> pd.Series:
    parsed, ratio = _try_parse_datetime_series(s)
    return parsed if ratio >= 0.4 else pd.to_datetime(s, errors="coerce")


def _looks_like_id(col: pd.Series) -> bool:
    n = len(col)
    if n < 30:
        return False
    nunique = col.nunique(dropna=True)
    if nunique == 0:
        return False
    return (nunique / max(1, n)) >= 0.85


def _is_mostly_numeric(col: pd.Series) -> bool:
    if pd.api.types.is_numeric_dtype(col):
        return True
    coerced = pd.to_numeric(col.astype(str).str.replace(",", "").str.strip(), errors="coerce")
    return coerced.notna().mean() >= 0.85


def _coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if pd.api.types.is_numeric_dtype(out[c]):
            continue
        if out[c].dtype == "object":
            s = out[c].astype(str).str.replace(",", "").str.strip()
            coerced = pd.to_numeric(s, errors="coerce")
            if coerced.notna().mean() >= 0.90:
                out[c] = coerced
    return out


def _infer_grain_from_dates(dt: pd.Series) -> str:
    u = dt.dropna().sort_values().unique()
    if len(u) < 3:
        return "unknown"
    deltas = np.diff(u).astype("timedelta64[D]").astype(int)
    if len(deltas) == 0:
        return "unknown"
    med = int(np.median(deltas))
    if med <= 2:
        return "daily"
    if med <= 10:
        return "weekly-ish"
    if med <= 40:
        return "monthly-ish"
    if med <= 120:
        return "quarterly-ish"
    return "yearly-ish"


def _format_pct(x: float) -> str:
    if x is None or (isinstance(x, float) and (np.isnan(x) or np.isinf(x))):
        return "-"
    return f"{x * 100:.1f}%"


def _ranked_contribution(df: pd.DataFrame, seg_col: str, metric_col: str) -> pd.DataFrame:
    tmp = df.groupby(seg_col, dropna=False)[metric_col].sum().sort_values(ascending=False)
    out = tmp.reset_index().rename(columns={metric_col: "value"})
    total = out["value"].sum()
    out["share"] = out["value"] / total if total != 0 else np.nan
    out["cum_share"] = out["share"].cumsum()
    return out


def _detect_anomalies(values: pd.Series, z_thresh: float = 3.0) -> pd.DataFrame:
    x = values.astype(float).copy()
    med = np.nanmedian(x)
    mad = np.nanmedian(np.abs(x - med))
    if mad == 0 or np.isnan(mad):
        return pd.DataFrame(columns=["idx", "value", "score", "type"])
    robust_z = 0.6745 * (x - med) / mad
    idx = np.where(np.abs(robust_z) >= z_thresh)[0]
    out = pd.DataFrame({"idx": idx, "value": x.iloc[idx].values, "score": robust_z.iloc[idx].values})
    out["type"] = np.where(out["score"] > 0, "spike", "drop")
    return out


def _make_insight(priority: int, title: str, evidence: str, action: str, tags: List[str]) -> dict:
    return {
        "Priority": priority,
        "Insight": title,
        "Evidence": evidence,
        "Suggested action": action,
        "Tags": ", ".join(tags),
    }


def _read_uploaded_file(file) -> pd.DataFrame:
    ext = _file_ext(file.name)
    if ext not in ALLOWED_EXT:
        raise ValueError(f"Unsupported file type: .{ext}")

    if ext == "csv":
        content = file.getvalue()
        for enc in ["utf-8", "utf-8-sig", "latin1"]:
            try:
                return pd.read_csv(io.BytesIO(content), encoding=enc)
            except Exception:
                continue
        return pd.read_csv(io.BytesIO(content))

    if ext in {"xlsx", "xls"}:
        return pd.read_excel(file)

    if ext == "parquet":
        return pd.read_parquet(file)

    raise ValueError("Unsupported file type.")


# ============================
# Validation + Capabilities
# ============================
@dataclass
class CapabilityFlags:
    trend_ready: bool
    segment_ready: bool
    anomaly_ready: bool
    driver_ready: bool
    investor_ready: bool
    llm_ready: bool


@dataclass
class ValidationResult:
    ok: bool
    errors: List[str]
    warnings: List[str]
    info: List[str]
    detected_date_cols: List[str]
    detected_metric_cols: List[str]
    detected_dimension_cols: List[str]
    suspected_id_cols: List[str]
    health_score: int
    capabilities: CapabilityFlags
    dataset_type: str


def _detect_cols_with_keywords(cols: List[str], keywords: List[str]) -> List[str]:
    if not keywords:
        return []
    out = []
    for c in cols:
        cl = c.lower().replace(" ", "_")
        if any(k in cl for k in keywords):
            out.append(c)
    return out


def _profile_validate_and_capabilities(df: pd.DataFrame, dataset_type: str) -> ValidationResult:
    errors, warnings, info = [], [], []

    if df is None or df.empty:
        caps = CapabilityFlags(False, False, False, False, False, False)
        return ValidationResult(
            ok=False,
            errors=["Dataset is empty."],
            warnings=[],
            info=[],
            detected_date_cols=[],
            detected_metric_cols=[],
            detected_dimension_cols=[],
            suspected_id_cols=[],
            health_score=0,
            capabilities=caps,
            dataset_type=dataset_type,
        )

    df = df.copy()
    df.columns = [_clean_colname(c) for c in df.columns]
    cols = list(df.columns)

    n_rows, n_cols = df.shape
    info.append(f"Rows: {n_rows:,} | Columns: {n_cols:,}")

    df2 = _coerce_numeric(df)

    missing_pct = (df2.isna().mean() * 100).sort_values(ascending=False)
    high_missing = missing_pct[missing_pct > 60]
    if len(high_missing) > 0:
        warnings.append(f"{len(high_missing)} columns have >60% missing values (may weaken insights).")

    dup_count = int(df2.duplicated().sum())
    if dup_count > 0:
        warnings.append(f"Detected {dup_count:,} duplicate rows (consider deduping).")

    rules = DATASET_RULES.get(dataset_type, DATASET_RULES["Generic"])

    # Detect time/date columns
    detected_date_cols: List[str] = []
    date_ratios: Dict[str, float] = {}
    preferred_time = set([c.lower() for c in rules.get("time_candidates", [])])

    for c in cols:
        cname = c.lower().strip()
        name_hint = (
            cname in COMMON_DATE_NAMES
            or "date" in cname
            or "month" in cname
            or "period" in cname
            or "time" in cname
            or cname in preferred_time
        )

        if name_hint:
            parsed, ratio = _try_parse_datetime_series(df2[c])
            date_ratios[c] = ratio
            if ratio >= 0.70:
                detected_date_cols.append(c)
            continue

        if df2[c].dtype == "object":
            sample = df2[c].dropna().astype(str).str.strip().head(50)
            if not sample.empty:
                patt_score = sample.str.contains(r"\d{1,4}[-/]\d{1,2}([-/]\d{1,2})?").mean()
                if patt_score >= 0.6:
                    parsed, ratio = _try_parse_datetime_series(df2[c])
                    date_ratios[c] = ratio
                    if ratio >= 0.70:
                        detected_date_cols.append(c)

    numeric_cols = [c for c in cols if pd.api.types.is_numeric_dtype(df2[c]) or _is_mostly_numeric(df2[c])]

    metric_by_keywords = _detect_cols_with_keywords(numeric_cols, rules.get("metric_keywords", []))
    metric_by_common = _detect_cols_with_keywords(numeric_cols, list(COMMON_METRIC_HINTS))

    detected_metric_cols: List[str] = []
    detected_metric_cols.extend(metric_by_keywords)
    for c in metric_by_common:
        if c not in detected_metric_cols:
            detected_metric_cols.append(c)

    if len(detected_metric_cols) == 0 and len(numeric_cols) > 0:
        detected_metric_cols = numeric_cols[: min(10, len(numeric_cols))]
        warnings.append("No obvious metric columns by name; using numeric columns as metric candidates.")

    suspected_id_cols: List[str] = []
    detected_dimension_cols: List[str] = []
    for c in cols:
        if c in detected_date_cols or c in numeric_cols:
            continue
        if _looks_like_id(df2[c]):
            suspected_id_cols.append(c)
            continue
        nunique = df2[c].nunique(dropna=True)
        if nunique <= 200 or (nunique / max(1, n_rows)) <= 0.05:
            detected_dimension_cols.append(c)

    # Dataset-specific soft warning: missing entity column for certain types
    entity_candidates = rules.get("entity_candidates", [])
    entity_found = False
    if entity_candidates:
        entity_found = any(any(ec in c.lower().replace(" ", "_") for ec in entity_candidates) for c in cols)

    time_found = len(detected_date_cols) > 0
    metric_found = len(detected_metric_cols) > 0
    dim_found = len(detected_dimension_cols) > 0

    if not metric_found:
        errors.append("No usable numeric metric column detected (needed for insights).")

    if dataset_type in {"ESG/Impact", "Portfolio Monitoring", "Startup/Deal Screening"} and not entity_found:
        warnings.append("No clear entity column detected (e.g., company/startup). Insights may be less meaningful.")

    # Health score
    score = 100
    score -= int(min(40, missing_pct.mean()))
    if dup_count > 0:
        score -= 5
    if len(high_missing) > 0:
        score -= 10
    if not time_found:
        score -= 10
    if not dim_found:
        score -= 10
    if not metric_found:
        score -= 40
    score = max(0, min(100, score))

    if time_found:
        best = sorted(detected_date_cols, key=lambda c: date_ratios.get(c, 0), reverse=True)[0]
        parsed, _ = _try_parse_datetime_series(df2[best])
        if parsed.notna().any():
            info.append(f"Best time column: '{best}' | Range: {parsed.min().date()} → {parsed.max().date()}")

    info.append(f"Health score: {score}/100")

    caps = CapabilityFlags(
        trend_ready=bool(time_found and metric_found),
        segment_ready=bool(dim_found),
        anomaly_ready=bool(time_found and metric_found),
        driver_ready=bool(dim_found and metric_found),
        investor_ready=bool(metric_found and (time_found or dim_found) and score >= 60),
        llm_ready=True,
    )

    return ValidationResult(
        ok=(len(errors) == 0),
        errors=errors,
        warnings=warnings,
        info=info,
        detected_date_cols=detected_date_cols,
        detected_metric_cols=detected_metric_cols,
        detected_dimension_cols=detected_dimension_cols,
        suspected_id_cols=suspected_id_cols,
        health_score=score,
        capabilities=caps,
        dataset_type=dataset_type,
    )


# ============================
# LLM Placeholder
# ============================
def build_llm_prompt(summary_payload: dict) -> str:
    return f"""
You are an analyst for uPledge. Write a clear narrative based ONLY on the provided summary JSON.
Do not invent data. If something is missing, say "unknown".

Return:
1) Executive summary (5 bullets)
2) Key insights (max 8)
3) Risks / data gaps (max 6)
4) Suggested next actions (max 6)
5) Questions to validate (max 6)

SUMMARY_JSON:
{summary_payload}
""".strip()


def call_llm_placeholder(provider: str, api_key: str, prompt: str) -> str:
    _ = (provider, api_key, prompt)
    return (
        "LLM Narrator (placeholder)\n\n"
        "Executive summary:\n"
        "- (Placeholder) Connect Gemini/OpenAI to generate real narrative.\n\n"
        "Key insights:\n"
        "- Insight 1...\n\n"
        "Risks / data gaps:\n"
        "- Gap 1...\n\n"
        "Suggested next actions:\n"
        "- Action 1...\n\n"
        "Questions to validate:\n"
        "- Question 1...\n"
    )


# ============================
# UI Helpers (Descriptions)
# ============================
def section_help(title: str, text: str):
    st.markdown(f"**{title}**")
    st.caption(text)


def badge_help():
    with st.expander("What do these badges mean?", expanded=False):
        st.markdown(
            """
- **Trend-ready**: time + metric exists → trends & growth can be computed
- **Driver-ready**: dimensions + metric → top contributors & movers
- **Segment-ready**: at least 1 dimension column detected
- **Anomaly-ready**: time + metric → spikes/drops can be flagged
- **Investor-ready**: dataset is structured enough for decision-style summary (not just exploratory)
- **LLM-ready**: narrative generation is possible if you enable it (uses computed summaries only)
"""
        )


# ============================
# Streamlit App
# ============================
st.set_page_config(page_title="uPledge — Data Summarizer (V2.1)", layout="wide")
st.title("uPledge — Data Summarizer with Advanced Insights (V2.1)")
st.caption("More user-friendly: each section includes a short explanation.")

with st.expander("Supported inputs (V2.1)", expanded=False):
    st.markdown(
        """
**Best inputs**
- CSV / Excel / Parquet (structured tables)
- Has: time column (date/month/period) + numeric metric (sales/revenue/score)
- Optional: dimensions (region/product/company/sector/channel)

**Not supported (yet)**
- PDFs, images, scanned reports, free-text documents (separate advanced tier)
"""
    )

# ---- Upload
section_help(
    "Upload dataset",
    "Upload a CSV/Excel/Parquet file. We’ll preview it, check data health, then generate insights based on your selections.",
)

colA, colB = st.columns([2, 1], gap="large")
with colA:
    uploaded = st.file_uploader("File upload", type=list(ALLOWED_EXT), accept_multiple_files=False)
with colB:
    section_help("Dataset type", "This helps uPledge apply better expectations (ESG vs business vs startup).")
    dataset_kind = st.selectbox("Dataset type", DATASET_OPTIONS, index=0)
    strict_time = st.toggle("Strict: require a time column (for trends)", value=False)
    preview_rows = st.slider("Preview rows", 20, 500, MAX_ROWS_PREVIEW_DEFAULT, step=10)

st.divider()

if not uploaded:
    st.info("Upload a file to begin.")
    st.stop()

# File size warning
try:
    size_mb = _bytes_to_mb(len(uploaded.getvalue()))
    if size_mb > MAX_MB_SOFT:
        st.warning(f"File size is {size_mb} MB. If it feels slow, try a smaller export or Parquet.")
except Exception:
    pass

# Read
try:
    df_in = _read_uploaded_file(uploaded)
except Exception as e:
    st.error(f"Failed to read file: {e}")
    st.stop()

# Preview
section_help("Preview", "Quickly confirm your data is uploaded correctly (columns align, values look right).")

left, right = st.columns([2, 1], gap="large")
with left:
    st.dataframe(df_in.head(preview_rows), use_container_width=True)
with right:
    section_help("Quick stats", "Basic dataset size + a column list to spot missing key columns.")
    st.write(f"**Rows:** {df_in.shape[0]:,}")
    st.write(f"**Columns:** {df_in.shape[1]:,}")
    st.write("**Columns list:**")
    st.code(", ".join([str(c) for c in df_in.columns[:60]]) + (" ..." if df_in.shape[1] > 60 else ""))

st.divider()

# Validate + Capabilities
section_help(
    "Validation + capability badges",
    "We check data health and show what uPledge can reliably compute (trends, drivers, anomalies).",
)

vr = _profile_validate_and_capabilities(df_in, dataset_kind)
caps = vr.capabilities

b1, b2, b3, b4 = st.columns([1, 1, 1, 1])
b1.metric("Health score", f"{vr.health_score}/100")
b2.metric("Time cols", str(len(vr.detected_date_cols)))
b3.metric("Metric cols", str(len(vr.detected_metric_cols)))
b4.metric("Dimensions", str(len(vr.detected_dimension_cols)))

badge_cols = st.columns(6)
badge_cols[0].metric("Trend-ready", "✅" if caps.trend_ready else "❌")
badge_cols[1].metric("Driver-ready", "✅" if caps.driver_ready else "❌")
badge_cols[2].metric("Segment-ready", "✅" if caps.segment_ready else "❌")
badge_cols[3].metric("Anomaly-ready", "✅" if caps.anomaly_ready else "❌")
badge_cols[4].metric("Investor-ready", "✅" if caps.investor_ready else "❌")
badge_cols[5].metric("LLM-ready", "✅" if caps.llm_ready else "❌")
badge_help()

for s in vr.info:
    st.write("• " + s)

if vr.suspected_id_cols:
    st.caption("Suspected ID columns (excluded from dimensions): " + ", ".join(vr.suspected_id_cols))

if vr.errors:
    st.error("Blocking issues (must fix):")
    for e in vr.errors:
        st.write("• " + e)

if vr.warnings:
    st.warning("Warnings (insights may be limited):")
    for w in vr.warnings:
        st.write("• " + w)

if strict_time and not caps.trend_ready:
    st.error("Strict mode is ON: a usable time/date/period column is required.")
    st.stop()

if not vr.ok:
    st.info("Fix the blocking issues, then re-upload (or export a cleaned version).")
    st.stop()

st.divider()

# Schema mapping
section_help(
    "Schema mapping (override)",
    "Select which columns represent time, your main metric, and the segments (dimensions) to compare.",
)

c1, c2, c3 = st.columns(3)
with c1:
    date_col = st.selectbox(
        "Time column",
        options=["(none)"] + vr.detected_date_cols,
        index=1 if vr.detected_date_cols else 0,
    )
with c2:
    metric_col = st.selectbox(
        "Primary metric (the number you want to explain)",
        options=vr.detected_metric_cols if vr.detected_metric_cols else list(df_in.columns),
        index=0,
    )
with c3:
    dims_default = vr.detected_dimension_cols[:5]
    dimensions = st.multiselect(
        "Dimensions (segments to compare)",
        options=[c for c in df_in.columns if c != metric_col],
        default=dims_default,
    )

st.caption("Example: Time=date, Metric=revenue, Dimensions=region/product/company/sector")

st.divider()

# LLM narrator (placeholder)
section_help(
    "LLM narrator (optional)",
    "If enabled, the LLM writes a plain-English narrative using computed summaries only (not raw rows).",
)
llm_on = st.toggle("Enable LLM narrator", value=False)
llm_provider = st.selectbox("Provider", ["ChatGPT (OpenAI)", "Gemini (Google)"], index=0, disabled=not llm_on)
llm_key = st.text_input("API Key", type="password", disabled=not llm_on)

st.divider()

run_insights = st.button("Generate insights", type="primary")

st.session_state["upledge_df"] = df_in
st.session_state["upledge_schema"] = {
    "dataset_kind": dataset_kind,
    "date_col": None if date_col == "(none)" else date_col,
    "metric_col": metric_col,
    "dimensions": dimensions,
    "validation": asdict(vr),
    "caps": asdict(vr.capabilities),
    "llm": {"enabled": llm_on, "provider": llm_provider, "has_key": bool(llm_key)},
}

if not run_insights:
    st.stop()

# ============================
# Insights Engine
# ============================
st.header("Insights Engine")
st.caption("This section generates insights based on the schema mapping above.")

df = df_in.copy()
date_col = st.session_state["upledge_schema"]["date_col"]
metric_col = st.session_state["upledge_schema"]["metric_col"]
dimensions = st.session_state["upledge_schema"]["dimensions"]

if metric_col not in df.columns:
    st.error("Selected metric column not found. Please reselect schema.")
    st.stop()

df.columns = [str(c).strip() for c in df.columns]
df[metric_col] = _to_numeric(df[metric_col])
df = df.dropna(subset=[metric_col])

if date_col and date_col in df.columns:
    df[date_col] = _safe_to_datetime(df[date_col])

with st.sidebar:
    st.subheader("Insights controls")
    st.caption("Tune sensitivity and top-N results.")
    z_thresh = st.slider("Anomaly sensitivity (Z threshold)", 2.0, 6.0, 3.0, 0.5)
    top_n = st.slider("Top-N for drivers", 3, 20, 8, 1)
    enable_downloads = st.toggle("Enable downloads", value=True)

tab1, tab2, tab3, tab4, tab5 = st.tabs(
    ["Executive Summary", "Trends", "Drivers", "Anomalies", "LLM Narrative"]
)

# Executive Summary
with tab1:
    section_help(
        "Executive Summary",
        "A quick, factual overview: dataset size, missingness, totals, and time coverage (if available).",
    )

    n_rows, n_cols = df_in.shape
    missing_overall = float(df_in.isna().mean().mean())
    metric_missing = float(df_in[metric_col].isna().mean()) if metric_col in df_in.columns else np.nan
    metric_total = float(df[metric_col].sum()) if len(df) else np.nan

    st.write(f"• Dataset type: **{dataset_kind}**")
    st.write(f"• Rows: **{n_rows:,}** | Columns: **{n_cols:,}**")
    st.write(f"• Missingness overall: **{missing_overall*100:.1f}%** | '{metric_col}' missing: **{metric_missing*100:.1f}%**")
    if not np.isnan(metric_total):
        st.write(f"• Total '{metric_col}': **{metric_total:,.2f}**")

    if caps.trend_ready and date_col and df[date_col].notna().any():
        dt = df[date_col].dropna()
        st.write(f"• Time coverage: **{dt.min().date()} → {dt.max().date()}**")
        st.write(f"• Time grain: **{_infer_grain_from_dates(dt)}**")
    else:
        st.write("• Trends are disabled (no usable time column selected).")

# Trends
with tab2:
    section_help(
        "Trends",
        "Shows how your metric changes over time (plus a smoothed rolling average). Requires a time column + metric.",
    )

    if not (caps.trend_ready and date_col and date_col in df.columns and df[date_col].notna().any()):
        st.info("Trend-ready is ❌ — trends are disabled.")
    else:
        ts = df.groupby(date_col)[metric_col].sum().sort_index()
        ts_df = ts.reset_index().rename(columns={date_col: "date", metric_col: "value"})
        ts_df["value"] = ts_df["value"].astype(float)
        ts_df["pct_change"] = ts_df["value"].pct_change()
        ts_df["rolling_mean_3"] = ts_df["value"].rolling(3, min_periods=1).mean()

        if len(ts_df) >= 2:
            latest = ts_df["value"].iloc[-1]
            prev = ts_df["value"].iloc[-2]
            mom = (latest - prev) / prev if prev not in (0, np.nan) else np.nan
            st.write(f"Latest: **{latest:,.2f}** | Previous: **{prev:,.2f}** | Change: **{_format_pct(mom)}**")

        st.line_chart(ts_df.set_index("date")[["value"]])
        st.line_chart(ts_df.set_index("date")[["rolling_mean_3"]])

        with st.expander("Show trend table"):
            show = ts_df.copy()
            show["pct_change"] = show["pct_change"].apply(lambda x: _format_pct(x) if pd.notna(x) else "-")
            st.dataframe(show, use_container_width=True)

# Drivers
with tab3:
    section_help(
        "Drivers",
        "Ranks which segments contribute most to the metric (and what changed most recently if time exists). Requires dimensions + metric.",
    )

    driver_insights: List[dict] = []

    if not caps.driver_ready or not dimensions:
        st.info("Driver-ready is ❌ or no dimensions selected — driver analysis is limited.")
    else:
        for dim in dimensions:
            if dim not in df.columns:
                continue

            st.markdown(f"### Dimension: `{dim}`")
            st.caption("Contribution table: value = total, share = % of total, cum_share = cumulative %.")
            contrib = _ranked_contribution(df, dim, metric_col).head(top_n)
            st.dataframe(contrib, use_container_width=True)

            if len(contrib) >= 3 and contrib["cum_share"].iloc[-1] >= 0.8:
                top_name = str(contrib[dim].iloc[0])
                top_share = float(contrib["share"].iloc[0])
                driver_insights.append(
                    _make_insight(
                        2,
                        f"High dependence on {dim}='{top_name}'",
                        f"Top {dim} contributes {_format_pct(top_share)} of '{metric_col}' (top {len(contrib)} contribute {contrib['cum_share'].iloc[-1]*100:.1f}%).",
                        f"Check concentration risk; consider balancing across other {dim} if needed.",
                        ["concentration", "risk", dim],
                    )
                )

            if caps.trend_ready and date_col and date_col in df.columns and df[date_col].notna().any():
                tmp = df.dropna(subset=[date_col]).copy()
                if tmp[date_col].nunique() >= 2:
                    last_date = tmp[date_col].max()
                    prev_date = tmp.loc[tmp[date_col] < last_date, date_col].max()

                    now = tmp[tmp[date_col] == last_date].groupby(dim)[metric_col].sum()
                    prev = tmp[tmp[date_col] == prev_date].groupby(dim)[metric_col].sum()

                    movers = (now - prev).sort_values(ascending=False).dropna()
                    st.caption(f"Top movers: which segments changed most ({prev_date.date()} → {last_date.date()}).")

                    movers_df = movers.head(top_n).reset_index()
                    movers_df.columns = [dim, "delta"]
                    st.dataframe(movers_df, use_container_width=True)

    st.session_state["upledge_driver_insights"] = driver_insights

# Anomalies
with tab4:
    section_help(
        "Anomalies",
        "Flags unusual spikes/drops in your metric over time. This is a signal to investigate, not a final explanation.",
    )

    anomalies_insights: List[dict] = []

    if not (caps.anomaly_ready and date_col and date_col in df.columns and df[date_col].notna().any()):
        st.info("Anomaly-ready is ❌ — anomaly detection is disabled.")
    else:
        ts = df.groupby(date_col)[metric_col].sum().sort_index()
        ts_df = ts.reset_index().rename(columns={date_col: "date", metric_col: "value"})
        ts_df["value"] = ts_df["value"].astype(float)

        scores = _detect_anomalies(ts_df["value"], z_thresh=z_thresh)
        if scores.empty:
            st.write("No strong anomalies detected at current sensitivity.")
        else:
            scores["date"] = ts_df.loc[scores["idx"], "date"].values
            scores = scores[["date", "value", "score", "type"]].sort_values("score", ascending=False)
            st.dataframe(scores, use_container_width=True)

            for _, r in scores.head(5).iterrows():
                d = r["date"]
                val = float(r["value"])
                typ = str(r["type"])
                sc = float(r["score"])
                anomalies_insights.append(
                    _make_insight(
                        1 if typ == "drop" else 2,
                        f"Anomalous {typ} detected on {pd.to_datetime(d).date()}",
                        f"Value: **{val:,.2f}** | Robust Z: **{sc:.2f}**",
                        "Investigate pipeline + business context (stockouts, missing data, one-off event).",
                        ["anomaly", typ],
                    )
                )

        st.line_chart(ts_df.set_index("date")[["value"]])

    st.session_state["upledge_anomaly_insights"] = anomalies_insights

# LLM Narrative (placeholder)
with tab5:
    section_help(
        "LLM Narrative (optional)",
        "Generates a plain-English narrative using computed summaries only. This is currently a placeholder output until you connect an API.",
    )

    if not llm_on:
        st.info("Enable the LLM narrator above to generate a narrative.")
    else:
        summary_payload = {
            "dataset_type": dataset_kind,
            "health_score": vr.health_score,
            "capabilities": asdict(vr.capabilities),
            "schema": {"time_col": date_col, "metric_col": metric_col, "dimensions": dimensions},
            "top_insights": (st.session_state.get("upledge_driver_insights", []) + st.session_state.get("upledge_anomaly_insights", []))[:10],
        }
        prompt = build_llm_prompt(summary_payload)
        provider_short = "openai" if "ChatGPT" in llm_provider else "gemini"

        narrative = call_llm_placeholder(provider_short, llm_key, prompt) if llm_key else call_llm_placeholder(provider_short, "", prompt)

        st.text_area("Narrative output", value=narrative, height=360)
        with st.expander("Prompt sent to LLM (debug)"):
            st.code(prompt)

# Export
st.divider()
section_help(
    "Export",
    "Download the ranked insights table (CSV) or a simple Markdown report for sharing with stakeholders.",
)

actionable = []
actionable.extend(st.session_state.get("upledge_driver_insights", []))
actionable.extend(st.session_state.get("upledge_anomaly_insights", []))

if actionable:
    out = pd.DataFrame(actionable).sort_values(["Priority"]).reset_index(drop=True)
    st.dataframe(out, use_container_width=True)

    if enable_downloads:
        st.download_button(
            "Download insights (CSV)",
            data=out.to_csv(index=False).encode("utf-8"),
            file_name="upledge_insights.csv",
            mime="text/csv",
        )

        md_lines = []
        md_lines.append("# uPledge — Data Summary & Insights\n")
        md_lines.append(f"- Dataset type: **{dataset_kind}**\n")
        md_lines.append(f"- Health score: **{vr.health_score}/100**\n")
        md_lines.append(f"- Metric: **{metric_col}**\n")
        md_lines.append(f"- Time column: **{date_col if date_col else '(none)'}**\n")
        md_lines.append(f"- Dimensions: **{', '.join(dimensions) if dimensions else '(none)'}**\n")
        md_lines.append("\n## Capability Badges\n")
        md_lines.append(f"- Trend-ready: {'YES' if caps.trend_ready else 'NO'}\n")
        md_lines.append(f"- Driver-ready: {'YES' if caps.driver_ready else 'NO'}\n")
        md_lines.append(f"- Segment-ready: {'YES' if caps.segment_ready else 'NO'}\n")
        md_lines.append(f"- Anomaly-ready: {'YES' if caps.anomaly_ready else 'NO'}\n")
        md_lines.append(f"- Investor-ready: {'YES' if caps.investor_ready else 'NO'}\n")

        md_lines.append("\n## Ranked Actionable Insights\n")
        for _, row in out.iterrows():
            md_lines.append(f"### (P{row['Priority']}) {row['Insight']}\n")
            md_lines.append(f"- Evidence: {row['Evidence']}\n")
            md_lines.append(f"- Suggested action: {row['Suggested action']}\n")
            md_lines.append(f"- Tags: {row['Tags']}\n")

        st.download_button(
            "Download report (Markdown)",
            data="\n".join(md_lines).encode("utf-8"),
            file_name="upledge_report.md",
            mime="text/markdown",
        )
else:
    st.info("No actionable insights generated yet. Select dimensions and/or ensure trend/anomaly readiness.")

st.caption("Next: replace LLM placeholder with real Gemini/OpenAI calls + add dataset-specific insight logic per option.")


Writing app.py


In [3]:
!pkill -f streamlit || true
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 \
  --server.headless true --server.enableCORS false --server.enableXsrfProtection false \
  &>/content/streamlit.log &


^C


In [4]:
!curl -I http://127.0.0.1:8501 | head


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to 127.0.0.1 port 8501 after 0 ms: Connection refused


In [5]:
from google.colab import output
output.serve_kernel_port_as_window(8501)
print("Proceess done ✅")
print("Please use Chrome browser to use the web app link above")


Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Proceess done ✅
Please use Chrome browser to use the web app link above


<div style="text-align:center; margin-top:40px;">

  <h2>🚀 Want to Go Further?</h2>

  <p style="font-size:16px; max-width:600px; margin:0 auto 24px;">
    This notebook is just a starting point.
    If you’d like to explore this with your own data or in a real-world setup, we’re happy to help.
  </p>

  <a href="https://upledge.io/contact-us"
     target="_blank"
     rel="noopener noreferrer"
     style="
       display:inline-block;
       padding:14px 28px;
       background-color:#2563EB;
       color:#ffffff;
       text-decoration:none;
       border-radius:8px;
       font-size:16px;
       font-weight:600;
       box-shadow:0 4px 10px rgba(0,0,0,0.15);
     ">
     Contact Us
  </a>

</div>
